In [16]:
import pandas as pd
import numpy as np
import os

In [17]:
directory = '/kaggle/input/raw-data/BTC/'

# Convert units K, M, B to number
def convert_to_number(x):
    if isinstance(x, str):
        if x.endswith('K'):
            return float(x[:-1]) * 1e3
        elif x.endswith('M'):
            return float(x[:-1]) * 1e6
        elif x.endswith('B'):
            return float(x[:-1]) * 1e9
        else:
            try:
                return float(x)
            except ValueError:
                return np.nan
    return x

# Process CSV file
def process_csv(file_path):
    try:
        df = pd.read_csv(file_path, header=None)
        df.columns = ['Date', 'Open_label', 'Open', 'High_label', 'High', 'Low_label', 'Low', 
                      'Close_label', 'Close', 'Change_label', 'Change', 'Amplitude_label', 'Amplitude',
                      'MA7_label', 'MA7', 'MA25_label', 'MA25', 'MA99_label', 'MA99',
                      'VolBTC_label', 'VolBTC', 'VolUSDT_label', 'VolUSDT', 'Unknown1', 'Unknown2']
        
        # Select and rename columns
        df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Amplitude', 'MA7', 'MA25', 'MA99', 'VolBTC', 'VolUSDT']]
        df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Amplitude', 'MA(7)', 'MA(25)', 'MA(99)', 'Vol(BTC)', 'Vol(USDT)']
        
        # Process data
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df['Change'] = df['Change'].str.rstrip('%').astype('float') / 100.0
        df['Amplitude'] = df['Amplitude'].str.rstrip('%').astype('float') / 100.0
        df['Vol(BTC)'] = df['Vol(BTC)'].apply(convert_to_number)
        df['Vol(USDT)'] = df['Vol(USDT)'].apply(convert_to_number)
        
        return df.dropna()
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return pd.DataFrame()

# Check for missing dates
def check_missing_dates(df, start_date, end_date):
    date_range = pd.date_range(start=start_date, end=end_date)
    missing_dates = date_range.difference(df['Date'])
    return missing_dates

# Handle missing dates using linear interpolation
def handle_missing_dates(df, missing_dates):
    # Create a DataFrame for missing dates with NaN values
    missing_df = pd.DataFrame(missing_dates, columns=['Date'])
    # Merge with the original DataFrame
    df_with_missing = pd.merge(df, missing_df, on='Date', how='outer', sort=True)
    # Interpolate missing values using linear method
    df_with_missing = df_with_missing.set_index('Date').interpolate(method='linear').reset_index()
    
    return df_with_missing


In [18]:

# Initialize a list to store DataFrames
dfs = []

# Iterate through all CSV files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        df = process_csv(file_path)
        if not df.empty:
            dfs.append(df)

# Combine all DataFrames
combined_df = pd.concat(dfs, ignore_index=True)

# Sort DataFrame by date
combined_df = combined_df.sort_values('Date')

# Filter data by date range
start_date = '2020-01-01'
end_date = '2024-01-01'
filtered_df = combined_df[(combined_df['Date'] >= start_date) & (combined_df['Date'] < end_date)]

# Display result
print("Filtered DataFrame:")
print(filtered_df.head())


Filtered DataFrame:
           Date     Open    High      Low    Close  Change  Amplitude  \
1230 2020-01-01  7195.24  7255.0  7175.15  7200.85  0.0008     0.0111   
1231 2020-01-02  7200.77  7212.5  6924.74  6965.71 -0.0327     0.0400   
1232 2020-01-03  6965.49  7405.0  6871.04  7344.96  0.0544     0.0767   
1233 2020-01-04  7345.00  7404.0  7272.21  7354.11  0.0012     0.0179   
1234 2020-01-05  7354.19  7495.0  7318.00  7358.75  0.0006     0.0241   

        MA(7)   MA(25)   MA(99)  Vol(BTC)    Vol(USDT)  
1230  7257.60  7210.74  7984.91   16792.0  121214000.0  
1231  7223.84  7188.96  7970.12   31951.0  225982000.0  
1232  7236.73  7189.22  7962.86   68429.0  495099000.0  
1233  7242.16  7194.41  7954.54   29988.0  219874000.0  
1234  7237.94  7200.36  7946.05   38331.0  284849000.0  


In [20]:
# Check for missing dates
missing_dates = check_missing_dates(filtered_df, start_date, end_date)

print("\nMissing dates:")
if len(missing_dates) > 0:
    print(missing_dates)
else:
    print("No missing dates found.")



Missing dates:
DatetimeIndex(['2020-01-08', '2020-01-20', '2020-02-02', '2020-02-14',
               '2020-02-26', '2020-03-09', '2020-03-21', '2020-04-02',
               '2020-04-14', '2020-04-27', '2020-05-09', '2020-05-21',
               '2020-06-02', '2020-06-14', '2020-06-26', '2020-07-08',
               '2020-07-21', '2020-08-02', '2020-08-14', '2020-08-26',
               '2020-09-07', '2020-09-19', '2020-10-01', '2020-10-14',
               '2020-10-26', '2020-11-07', '2020-11-19', '2020-12-01',
               '2020-12-13', '2020-12-25', '2021-01-07', '2021-01-19',
               '2021-01-31', '2021-02-21', '2021-03-22', '2021-04-19',
               '2021-05-18', '2021-06-15', '2021-07-14', '2021-08-11',
               '2021-09-09', '2021-10-07', '2021-11-05', '2021-12-03',
               '2022-01-01', '2024-01-01'],
              dtype='datetime64[ns]', freq=None)


In [21]:
filled_df = handle_missing_dates(filtered_df, missing_dates)
print("\nDataFrame after handling missing dates:")
print(filled_df.head())


DataFrame after handling missing dates:
        Date     Open    High      Low    Close  Change  Amplitude    MA(7)  \
0 2020-01-01  7195.24  7255.0  7175.15  7200.85  0.0008     0.0111  7257.60   
1 2020-01-02  7200.77  7212.5  6924.74  6965.71 -0.0327     0.0400  7223.84   
2 2020-01-03  6965.49  7405.0  6871.04  7344.96  0.0544     0.0767  7236.73   
3 2020-01-04  7345.00  7404.0  7272.21  7354.11  0.0012     0.0179  7242.16   
4 2020-01-05  7354.19  7495.0  7318.00  7358.75  0.0006     0.0241  7237.94   

    MA(25)   MA(99)  Vol(BTC)    Vol(USDT)  
0  7210.74  7984.91   16792.0  121214000.0  
1  7188.96  7970.12   31951.0  225982000.0  
2  7189.22  7962.86   68429.0  495099000.0  
3  7194.41  7954.54   29988.0  219874000.0  
4  7200.36  7946.05   38331.0  284849000.0  


In [22]:
missing_dates2 = check_missing_dates(filled_df, start_date, end_date)

print("\nMissing dates:")
if len(missing_dates2) > 0:
    print(missing_dates2)
else:
    print("No missing dates found.")


Missing dates:
No missing dates found.


In [25]:
BTC_data = filled_df
BTC_data.head(10)

,Date,Open,High,Low,Close,Change,Amplitude,MA(7),MA(25),MA(99),Vol(BTC),Vol(USDT)
0,2020-01-01,7195.24,7255.00,7175.150,7200.85,0.00080,0.0111,7257.60,7210.740,7984.91,16792.0,121214000.0
1,2020-01-02,7200.77,7212.50,6924.740,6965.71,-0.03270,0.0400,7223.84,7188.960,7970.12,31951.0,225982000.0
2,2020-01-03,6965.49,7405.00,6871.040,7344.96,0.05440,0.0767,7236.73,7189.220,7962.86,68429.0,495099000.0
3,2020-01-04,7345.00,7404.00,7272.210,7354.11,0.00120,0.0179,7242.16,7194.410,7954.54,29988.0,219874000.0
4,2020-01-05,7354.19,7495.00,7318.000,7358.75,0.00060,0.0241,7237.94,7200.360,7946.05,38331.0,284849000.0
5,2020-01-06,7357.64,7795.34,7346.760,7758.00,0.05430,0.0610,7311.09,7222.760,7943.17,54636.0,413330000.0
6,2020-01-07,7758.90,8207.68,7723.710,8145.28,0.04990,0.0624,7446.81,7258.230,7941.71,91172.0,724203000.0
7,2020-01-08,7906.81,8131.82,7736.855,7981.52,0.01015,0.0502,7568.75,7292.055,7937.78,77706.0,615730500.0
8,2020-01-09,8054.72,8055.96,7750.000,7817.76,-0.02960,0.0380,7690.69,7325.880,7933.85,64240.0,507258000.0
9,2020-01-10,7817.74,8199.00,7672.000,8197.02,0.04850,0.0674,7812.41,7378.090,7933.57,82407.0,652774000.0


In [24]:
BTC_data.to_csv('/kaggle/working/BTC_data.csv', index=False)  